### Layers : Fundamental blocks of Neural Network

In [1]:
import torch
from torch.nn import Linear, ReLU
import torch.nn as nn
import numpy as np
from torch.autograd import Variable



### Linear(in_features,out_features,bias)

定義 輸入和輸出層的維度(weight的維度)

In [2]:
myLayer = Linear(in_features=10,out_features=5,bias=True)
inp = Variable(torch.randn(1,10))
myLayer = Linear(in_features=10,out_features=5,bias=True) 
myLayer(inp)

tensor([[-0.5043, -0.9452, -0.2227,  0.6060, -0.0637]],
       grad_fn=<AddmmBackward>)

In [3]:
myLayer.weight

Parameter containing:
tensor([[-0.0945,  0.2008, -0.1593,  0.1647,  0.1253, -0.3072,  0.2887, -0.1306,
         -0.1266, -0.0958],
        [-0.0722,  0.0916, -0.2821,  0.0060,  0.0675, -0.1782, -0.0556, -0.3038,
         -0.1205, -0.2765],
        [-0.1606,  0.2588, -0.3080,  0.2579,  0.1673,  0.1432,  0.0645, -0.2425,
         -0.1204, -0.0503],
        [-0.2912, -0.0441,  0.2624,  0.0130,  0.2188, -0.0714,  0.3049,  0.2062,
          0.0970,  0.0201],
        [-0.1975,  0.1083, -0.0462, -0.1595, -0.0147,  0.0779,  0.1128, -0.0493,
          0.0009, -0.0293]], requires_grad=True)

In [4]:
myLayer.bias

Parameter containing:
tensor([-0.0160, -0.0228, -0.0096, -0.0321, -0.0408], requires_grad=True)

### Stacking Linear layers

建立多層神經層透過迭代

In [5]:
myLayer1 = Linear(10,5)
myLayer2 = Linear(5,2)
myLayer2(myLayer1(inp))

tensor([[-0.1608,  0.2106]], grad_fn=<AddmmBackward>)

### PyTorch Non-linear Activations

使用兩種呼叫激勵函數的不同方式

In [6]:
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
myRelu = ReLU()
myRelu(sample_data)


tensor([[1., 2., 0., 0.]])

In [7]:
import torch.nn as nn
import torch.nn.functional as F
sample_data = Variable(torch.Tensor([[1,2,-1,-1]])) 
f = F.relu(sample_data) # Much simpler.
f

tensor([[1., 2., 0., 0.]])

### Neural Network 

利用類別建立多層神經網路

In [8]:
class MyFirstNetwork(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(MyFirstNetwork,self).__init__() 
        self.layer1 = nn.Linear(input_size,hidden_size) 
        self.layer2 = nn.Linear(hidden_size,output_size)
    def layer(self):
        first  = [self.layer1.weight.data,self.layer1.bias.data]
        second = [self.layer2.weight.data,self.layer2.bias.data]
        return first,second
    def forward(self,input): 
        out = self.layer1(input) 
        out = F.relu(out)
        out = self.layer2(out) 
        out = F.softmax(out,dim=1)
        return out

model = MyFirstNetwork(112*112*3,56*56,2)
print(model)

MyFirstNetwork(
  (layer1): Linear(in_features=37632, out_features=3136, bias=True)
  (layer2): Linear(in_features=3136, out_features=2, bias=True)
)


### Loss

介紹兩種不同的loss function

In [9]:
loss = nn.MSELoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.randn(3, 5))
output = loss(input, target)
output.backward()

分類問題常用的loss function: 交叉熵(cross-entropy)

In [10]:
def cross_entropy(true_label, prediction):
    if true_label == 1:
        return -log(prediction)
    else:
        return -log(1 - prediction)

### loss(x,class)=−x[class]+log(∑exp(x[j]))

利用上述公式自訂函數與內建函數比較

In [11]:
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(3, 5), requires_grad=True) 
target = Variable(torch.LongTensor(3).random_(5)) 
output = loss(input, target)
output.backward()
print(output.data.item())

def CrossEntropyLoss(input,target):
    sum = 0
    for i in range(len(input)):
        tmp = 0
        for j in range(len(input[0])):
            tmp+=np.e**(input[i][j])
        sum+=-input[i][target[i]]+np.log(tmp)
    return float(sum)/len(input)

print(CrossEntropyLoss(input.data.numpy(),target.data.numpy()))

1.5371613502502441
1.5371613815822813


### Optimizer

將多個圖形利用自訂的類別建立成一個物件

In [12]:
from glob import glob
from PIL import Image
from torch.utils.data import Dataset
# ### Optimizer
class DogsAndCatsDataset(Dataset):
    def __init__(self,root_dir,size=(112,112)):
        self.files = glob(root_dir)
        self.size = size
        self.arr = np.array([[0],[1]],int)
    def __len__(self):
        return len(self.files)
    def __getitem__(self,idx):
        input = Variable(torch.from_numpy(np.asarray(Image.open(self.files[idx]).resize((self.size)), float).reshape(1,-1)))
        if(idx%2==0):
            target = Variable(torch.from_numpy(self.arr[0]))
        else:
            target = Variable(torch.from_numpy(self.arr[1])) 
        return input,target

dataset=DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/classifier/*.jpg")

### Training

In [13]:
import torch.optim as optim
from time import perf_counter
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
print("Training......")

t1 = perf_counter()
t  = t1
for i in range(10):
    index = 0
    print("epoch",i)
    for input, target in dataset:
        output = model(input.float()/(3*(255.)**2)**0.5)
        optimizer.zero_grad()
        loss = loss_fn(output, target.long())
        loss.backward()
        optimizer.step()
        index+=1
        if(perf_counter()-t > 30):
            t = perf_counter()
            print("  Completion ratio :",str(index*100.0/60)+"%")
t2  = perf_counter()

print(model)
print("--------------------------------------")
print("layer1 :","\nweight :\n",model.layer()[0][0].numpy(),"\nbias :\n",model.layer()[0][1].numpy())
print("")
print("layer2 :","\nweight :\n",model.layer()[1][0].numpy(),"\nbias :\n",model.layer()[1][1].numpy())
print("time :",str((t2-t1)/60.0)+" min")

print("\ntest.....")
test = DogsAndCatsDataset("D:/program/vscode_workspace/private/data/dogs-vs-cats/sample_test/*.jpg")
output1 = test[0][0]
output2 = test[1][0]
output1 = model(output1.float())
output2 = model(output2 .float())
print(output1)
print(output2)

Training......
epoch 0
  Completion ratio : 51.666666666666664%
epoch 1
  Completion ratio : 3.3333333333333335%
  Completion ratio : 55.0%
epoch 2
  Completion ratio : 5.0%
  Completion ratio : 58.333333333333336%
epoch 3
  Completion ratio : 8.333333333333334%
  Completion ratio : 61.666666666666664%
epoch 4
  Completion ratio : 11.666666666666666%
  Completion ratio : 65.0%
epoch 5
  Completion ratio : 13.333333333333334%
  Completion ratio : 61.666666666666664%
epoch 6
  Completion ratio : 6.666666666666667%
  Completion ratio : 55.0%
  Completion ratio : 98.33333333333333%
epoch 7
  Completion ratio : 45.0%
  Completion ratio : 88.33333333333333%
epoch 8
  Completion ratio : 35.0%
  Completion ratio : 80.0%
epoch 9
  Completion ratio : 25.0%
  Completion ratio : 70.0%
MyFirstNetwork(
  (layer1): Linear(in_features=37632, out_features=3136, bias=True)
  (layer2): Linear(in_features=3136, out_features=2, bias=True)
)
--------------------------------------
layer1 : 
weight :
 [[-5.17